In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install numpy pandas matplotlib seaborn opencv-python tensorflow scikit-learn scikit-image scipy

In [3]:
import os
import shutil
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [4]:
data_dir = '/content/drive/MyDrive/NHL University/Datasets/animal_data'
train_dir = '/content/drive/MyDrive/NHL University/Datasets/animal_data_spilited/train'
test_dir = '/content/drive/MyDrive/NHL University/Datasets/animal_data_spilited/test'

test_size_ratio = 0.2
classes = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]

for cls in classes:
    # Create corresponding directories in train and test
    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(test_dir, cls), exist_ok=True)

    # Get all files from the class directory
    files = [os.path.join(data_dir, cls, f) for f in os.listdir(os.path.join(data_dir, cls)) if os.path.isfile(os.path.join(data_dir, cls, f))]

    # Split the files
    train_files, test_files = train_test_split(files, test_size=test_size_ratio, random_state=42)

    # Move the files to the corresponding new directories
    for f in train_files:
        shutil.move(f, os.path.join(train_dir, cls))
    for f in test_files:
        shutil.move(f, os.path.join(test_dir, cls))

In [5]:
train_dir = '/content/drive/MyDrive/NHL University/Datasets/animal_data_spilited/train'
test_dir = '/content/drive/MyDrive/NHL University/Datasets/animal_data_spilited/test'
# Function to count files
def count_files(directory):
    return sum([len(files) for r, d, files in os.walk(directory)])

# Print counts
print("Files in training set:", count_files(train_dir))
print("Files in testing set:", count_files(test_dir))


Files in training set: 1549
Files in testing set: 395


In [6]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow images in batches using train_datagen generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # EfficientNet default input size
    batch_size=32,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False)

Found 1549 images belonging to 15 classes.
Found 395 images belonging to 15 classes.


In [7]:
# Load EfficientNetB0 base model
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
base_model.trainable = False

# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# Define the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()


16705208/16705208 [==============================] - 2s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 224, 224, 3)          0         ['input_1[0][0]']             
                                                                                                  
 normalization (Normalizati  (None, 224, 224, 3)          7         ['rescaling[0][0]']           
 on)                                                                                              
                                                                                                  
 rescaling_1 (Rescaling)     

In [8]:
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=test_generator)

Epoch 1/20
49/49 [==============================] - 1662s 34s/step - loss: 2.7885 - accuracy: 0.0516 - val_loss: 2.7276 - val_accuracy: 0.0709
Epoch 2/20
49/49 [==============================] - 9s 191ms/step - loss: 2.7203 - accuracy: 0.0523 - val_loss: 2.7088 - val_accuracy: 0.0684
Epoch 3/20
49/49 [==============================] - 9s 176ms/step - loss: 2.7110 - accuracy: 0.0626 - val_loss: 2.7089 - val_accuracy: 0.0633
Epoch 4/20
49/49 [==============================] - 9s 188ms/step - loss: 2.7092 - accuracy: 0.0620 - val_loss: 2.7079 - val_accuracy: 0.0658
Epoch 5/20
49/49 [==============================] - 10s 211ms/step - loss: 2.7086 - accuracy: 0.0581 - val_loss: 2.7079 - val_accuracy: 0.0709
Epoch 6/20
49/49 [==============================] - 10s 204ms/step - loss: 2.7082 - accuracy: 0.0633 - val_loss: 2.7079 - val_accuracy: 0.0709
Epoch 7/20
49/49 [==============================] - 9s 193ms/step - loss: 2.7092 - accuracy: 0.0665 - val_loss: 2.7078 - val_accuracy: 0.0709
Epo

In [9]:
test_loss, test_acc = model.evaluate(test_generator)
print("Test accuracy:", test_acc)

13/13 [==============================] - 2s 139ms/step - loss: 2.7076 - accuracy: 0.0709
Test accuracy: 0.07088607549667358


In [ ]:
from tensorflow.keras.preprocessing import image

def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Create a batch
    img_array /= 255.0  # Rescale
    return img_array

# Example usage
img_path = '/path_to_new_image.jpg'
img_array = load_and_preprocess_image(img_path)
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions, axis=1)
print(f"Predicted class: {train_generator.class_indices[predicted_class]}")
